# Импорты

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from transformers import GPT2Tokenizer, T5ForConditionalGeneration
from sentence_transformers import SentenceTransformer
from nltk.corpus import stopwords
import clickhouse_connect
import numpy as np
import torch
import os


# Реализация класса модели

In [4]:
torch.set_num_threads(4)

# Set to False if u havent downloaded pytorch_models to /RussiaBank/models/<filename>
LOCAL = True

MODEL_PATH = "FRED-T5-large"

if LOCAL:
    llm_model = T5ForConditionalGeneration.from_pretrained(os.path.dirname(os.path.realpath("")) + f"/RussiaBank/models/{MODEL_PATH}")
    tokenizer_model = GPT2Tokenizer.from_pretrained(os.path.dirname(os.path.realpath("")) + f"/RussiaBank/models//{MODEL_PATH}",eos_token='</s>')
else:
    llm_model = T5ForConditionalGeneration.from_pretrained("ai-forever/FRED-T5-large")
    tokenizer_model = GPT2Tokenizer.from_pretrained("ai-forever/FRED-T5-large",eos_token='</s>')

embedding_model = SentenceTransformer('intfloat/multilingual-e5-large-instruct')



class Model:
    def __init__(self, 
                 client,
                 llm_model = llm_model, 
                 tokenizer_model = tokenizer_model,
                 embedding_model = embedding_model) -> None:
        """
        Конструктор класса Model.

        Параметры:
        - client: объект клиента для работы с базой данных
        - llm_model: модель для генерации текста
        - tokenizer_model: токенизатор для модели
        - embedding_model: модель для встраивания текста

        Возвращаемое значение:
        Нет

        Пример использования:
        model = Model(client)
        """
        self.client = client
        self.llm_model = llm_model
        self.tokenizer_model = tokenizer_model
        self.embedding_model = embedding_model
        self.scores_by_documents = {}
        self.query = ""
        self.k_top_chunks_in_docs: list[tuple[int,int]] = []

    
    def get_row(self, document_id: int, chunk_number: int) -> list:
        """
        Метод для получения строки из базы данных по идентификатору документа и номеру части.

        Параметры:
        - document_id: идентификатор документа
        - chunk_number: номер части документа

        Возвращаемое значение:
        - Результат запроса к базе данных

        Пример использования:
        row = model.get_row(1, 2)
        """
        return self.client.command(f"SELECT * FROM test_table WHERE id = {document_id} AND chunk_number = {chunk_number}")

    
    def encode_query(self, query: str) -> list[float]:
        """
        Метод для кодирования текста запроса.
    
        Параметры:
        - query: str, текст запроса
    
        Возвращаемое значение:
        - list[float], закодированный текст запроса
    
        Пример использования:
        encoded_query = model.encode_query("текст запроса")
        """
        return self.embedding_model.encode(query)

    
    def calculate_similarities(self, query: str) -> None:
        """
        Метод для вычисления сходства между запросом и документами в базе данных.
    
        Параметры:
        - query: str, текст запроса
    
        Возвращаемое значение:
        Нет
    
        Пример использования:
        model.calculate_similarities("текст запроса")
        """
        self.query = query
        
        count_docs: int = self.client.command(f"SELECT COUNT(DISTINCT id) FROM test_table")
        
        encoded_query: list[float] = self.encode_query(query)

        ids_range = sorted(map(int, self.client.command(f"SELECT DISTINCT id FROM test_table").split("\n")))
        
        for id in range(1, count_docs + 1):
        
            self.scores_by_documents[id] = []
            
            count_chunks: int = self.client.command(f"SELECT COUNT(DISTINCT chunk_number) FROM test_table WHERE id = {id}")
            
            for chunk_num in range(1, count_chunks + 1):
            
                chunk: list[float] = list(map(float,self.client.command(f"SELECT chunk_embeding FROM test_table WHERE id = {id} AND chunk_number = {chunk_num}")[1:-1].split(",")))
                
                self.scores_by_documents[id].append(self.get_score_by_chunk(encoded_query, chunk))
                                                      
    def get_score_by_chunk(self, encoded_query, chunk):
        return chunk @ encoded_query / np.linalg.norm(chunk)

    
    def get_top_k_chunks(self, k_: int = 1) -> list[tuple]:
        """Метод для получения топ-K фрагментов из документов.
    
        Параметры:
        - k_: int, количество топ-фрагментов (по умолчанию 1)
        
        Возвращаемое значение:
        - list[tuple], список кортежей с индексами топ-фрагментов
        
        Пример использования:
        top_chunks = model.get_top_k_chunks(5)
        """
        all_chunk_scores: list[int] = []
        
        for i in list(self.scores_by_documents.values()):
        
            all_chunk_scores.extend(i)
        
        all_chunk_indices_scores_sorted: list[int] = np.argsort(all_chunk_scores)[::-1][:k_]

        dict_: dict[int, list[int]] = {}
        
        indx: int = 0
        
        for k in self.scores_by_documents:
        
            dict_[k] = []
            
            for i in range(len(self.scores_by_documents[k])):
            
                dict_[k].append(indx)
                
                indx += 1

        k_top_chunks_in_docs: list[tuple[int,int]] = []
        
        for key in dict_:
            
            for val in all_chunk_indices_scores_sorted:
            
                if val in dict_[key]:
                
                    temp: tuple[int,int] = (key,dict_[key].index(val))
                    
                    k_top_chunks_in_docs.append(temp)
        
        return k_top_chunks_in_docs

    
    def get_top_k_chunks_texts(self, k: int = 1) -> list[str]:
        """Метод для получения текстов топ-K фрагментов из документов.

        Параметры:
        - k: int, количество топ-фрагментов (по умолчанию 1)
    
        Возвращаемое значение:
        - list[str], список текстов топ-фрагментов
    
        Пример использования:
        top_chunks_texts = model.get_top_k_chunks_texts(5)
        """
        self.k_top_chunks_in_docs: list[tuple[int,int]] = self.get_top_k_chunks(k)
        
        res = []
        
        for i in range(k):
        
            id, chunk_number = self.k_top_chunks_in_docs[i][0], self.k_top_chunks_in_docs[i][1]
            
            res.append(self.client.command(f"SELECT chunk_text FROM test_table WHERE id = {id} AND chunk_number = {chunk_number+1}"))    
        
        return res

    
    def generate(self,ninp: str,p={
            "do_sample":True,
            "top_p":0.9,
            "temperature":0.05,
            "repetition_penalty": 4.0,
            "min_length": 5,
            "max_length": 70,
            "tokens_offset":0
            }) -> str:
        """Метод для генерации ответа на основе контекста с использованием модели T5.

        Параметры:
        - ninp: str, входной текстовый контекст
        - p: Dict[str, Union[bool, float, int]], параметры для генерации ответа (по умолчанию используются предустановленные значения)
    
        Возвращаемое значение:
        - str, сгенерированный ответ
    
        Пример использования:
        answer = model.generate("Важный текст для генерации ответа")
        """
        ninp = f"""Ответь пожалуйста на вопрос: '{self.query}' при том условии, что ты знаешь только эту информацию: '{ninp}'."""
        id, chunk_number = self.k_top_chunks_in_docs[0][0], self.k_top_chunks_in_docs[0][1]
        url = self.client.command(f"SELECT document_url FROM test_table WHERE id = {id} AND chunk_number = {chunk_number+1}")
        input_ids=torch.tensor([self.tokenizer_model.encode(ninp,add_special_tokens=True)])
        outputs=self.llm_model.generate(input_ids,do_sample = p["do_sample"],top_p = p["top_p"],temperature=p["temperature"],repetition_penalty = p["repetition_penalty"],min_length = p["min_length"],max_length = p["max_length"],eos_token_id=self.tokenizer_model.encode(']',add_special_tokens=True)[0])#tokenizer.eos_token_id 
        answer = f"URL документа: {url}.\n" + self.tokenizer_model.decode(outputs[0][1+p["tokens_offset"]:], skip_special_tokens=True)
        return answer


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
client = clickhouse_connect.get_client(host='localhost', port=8123, username='default', password='')
model = Model(client)


In [6]:
import pandas as pd

df = pd.read_csv("dataset.csv")
df.head()

,question,answer,url
0,Что понимается под EGS рейтингом?,под ESG-рейтингом (рейтингом устойчивого разви...,https://www.cbr.ru/Crosscut/LawActs/File/6225
1,фактор обусловливающих реализуемые организацие...,"Один из определяющих факторов, обусловливающих...",https://www.cbr.ru/Crosscut/LawActs/File/6225
2,как определить категорию egs htqnbyuf,Для определения категории ESG-рейтинга можно и...,https://www.cbr.ru/Crosscut/LawActs/File/6225
3,для чего разработан стандарт ФИНАНСОВЫЕ СООБЩЕ...,Стандарт разработан в целях организации обмена...,https://www.cbr.ru/Crosscut/LawActs/File/7647
4,Что говорится в Информационном письме Банка Ро...,Пунктом 4 части 1 статьи 7 Федерального закона...,https://www.cbr.ru/Crosscut/LawActs/File/7639


# Пример работы модели

In [ ]:
for question, answer in zip(df['question'], df['answer']):
    model.calculate_similarities(question)
    for text in model.get_top_k_chunks_texts()[:1]:
        print("_____________________________________________________________________")
        print(f"Вопрос: {question}\nЭталонный ответ: {answer}")
        print("_____________________________________________________________________")
        print(f"Ответ на вопрос от модели:\n{model.generate(text)}")

_____________________________________________________________________
Вопрос: Что понимается под EGS рейтингом?
Эталонный ответ: под ESG-рейтингом (рейтингом устойчивого развития) понимается выраженное публично независимое внешнее мнение о профиле рейтингуемого лица (организации или эмитента) – количественных и качественных характеристиках его деятельности в области устойчивого развития – на дату присвоения ESG-рейтинга, а также об управлении и подверженности рейтингуемого лица рискам устойчивого развития на горизонте от одного года до трех лет
_____________________________________________________________________
Ответ на вопрос от модели:
URL документа: https://www.cbr.ru/Crosscut/LawActs/File/6225.
 рейтинговых компаний в России и за рубежом.
'Введение общественное интерес быстрорастущему рынку рейтингов1 публичных оценок экологических социальных экономических характеристик организации финансового инструмента продукта финансовом инструменте профиле характеристиках компании подверженн